# 下载模型与数据集

In [ ]:
import os

# 设置镜像地址
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError


def download_model(target_dir, repo_id):
    """
    将指定的 HuggingFace 仓库下载到本地目录。
    
    参数:
        target_dir (str): 本地保存模型的路径。
        repo_id (str): HuggingFace 上的仓库 ID。
    """
    print(f"准备开始下载 '{repo_id}' ...")
    print(f"目标保存路径: {os.path.abspath(target_dir)}")

    try:
        # snapshot_download 会下载整个仓库
        # local_dir_use_symlinks=False 确保下载的是实际文件而不是缓存的软链接
        local_path = snapshot_download(
            repo_id=repo_id,
            local_dir=target_dir,
            local_dir_use_symlinks=False,
            resume_download=True,  # 支持断点续传
            endpoint="https://hf-mirror.com"
            # max_workers=4        # 可选：如果网速允许，增加并发数
        )
        print(f"✅ 下载成功！文件已保存在: {local_path}")

    except HfHubHTTPError as e:
        print(f"❌ 下载失败 (网络或权限错误): {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")


from datasets import load_dataset
import datasets


def download_dataset(target_dir, repo_id):
    """从Hugging Face上下载数据集到本地"""
    # 设置国内镜像加速
    os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

    # 检查并创建目录
    # exist_ok=True: 如果目录已存在，不会报错
    # parents=True: 如果父目录也不存在，会一并创建（递归创建）
    if not os.path.exists(target_dir):
        print(f"目标目录不存在，正在创建: {target_dir}")
        os.makedirs(target_dir, exist_ok=True)

    # 显式开启进度条显示
    datasets.enable_progress_bar()

    print(f"准备开始下载 '{repo_id}' ...")
    print(f"目标保存路径: {os.path.abspath(target_dir)}")

    try:
        # snapshot_download 会下载整个仓库
        # local_dir_use_symlinks=False 确保下载的是实际文件而不是缓存的软链接
        local_path = snapshot_download(
            repo_id=repo_id,
            local_dir=target_dir,
            local_dir_use_symlinks=False,
            resume_download=True,  # 支持断点续传
            # max_workers=4        # 可选：如果网速允许，增加并发数
            repo_type="dataset",  # 必须指定
            force_download=True,
            endpoint="https://hf-mirror.com",
        )
        print(f"✅ 下载成功！文件已保存在: {local_path}")

    except HfHubHTTPError as e:
        print(f"❌ 下载失败 (网络或权限错误): {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")



下载向量模型:Qwen/Qwen3-Embedding-8B

In [ ]:
target_dir = "/root/autodl-tmp/model/qwen3-8b"
repo_id = "Qwen/Qwen3-Embedding-8B"
download_model(target_dir=target_dir, repo_id=repo_id)

下载向量模型:Qwen/Qwen3-Embedding-4B

In [ ]:
target_dir = "/root/autodl-tmp/model/Qwen3-Embedding-4B"
repo_id = "Qwen/Qwen3-Embedding-4B"
download_model(target_dir=target_dir, repo_id=repo_id)

下载向量模型:Qwen/Qwen3-Embedding-0.6B，需要1.12 GB空间

In [ ]:
# !source /etc/network_turbo && hf download --repo-type model Qwen/Qwen3-Embedding-0.6B --local-dir /root/autodl-tmp/model/Qwen3-Embedding-0.6B --force-download
target_dir = "./model/Qwen3-Embedding-0.6B-1"
repo_id = "Qwen/Qwen3-Embedding-0.6B"
download_model(target_dir=target_dir, repo_id=repo_id)

下载识别模型：ds4sd/docling-models

In [ ]:
download_model(
    target_dir="/root/autodl-tmp/model/docling",
    repo_id="ds4sd/docling-models")

下载数据集：yubo2333/MMLongBench-Doc

In [ ]:
download_dataset(repo_id="yubo2333/MMLongBench-Doc", target_dir="./dataset/MMLongBench")

下载数据集：LongDocURL

In [ ]:
download_dataset(target_dir="")

下载NarrativeQA数据集

In [ ]:
download_dataset(repo_id="deepmind/narrativeqa", target_dir="/root/autodl-tmp/dataset")

下载QASPER数据集

In [ ]:
download_dataset(repo_id="allenai/qasper", target_dir="/root/autodl-tmp/dataset/qasper")

下载问答模型allenai/unifiedqa-v2-t5-3b-1363200

In [ ]:
download_model(
    target_dir="/root/autodl-tmp/model/unifiedqa",
    repo_id="allenai/unifiedqa-v2-t5-3b-1363200")

# Pipeline

In [ ]:
from pipeline import RunConfig, PipelineConfig, Pipeline

preprocess_configs = {"ser_tab": RunConfig(use_serialized_tables=True),
                      "no_ser_tab": RunConfig(use_serialized_tables=False)}

max_st_qwenturbo8k_reasoning_config = RunConfig(
    use_serialized_tables=False,
    parent_document_retrieval=True,
    llm_reranking=True,
    parallel_requests=1,
    submission_name="",
    pipeline_details="",
    api_provider="qwen",
    answering_model="qwen-turbo",
    config_suffix="_max_qwen-turbo1-llmre-reasoning"
)

In [ ]:
from pyprojroot import here

root_path = here() / "data" / "test_set"
print("root_path:", root_path)
pipeline = Pipeline(root_path,
                    run_config=max_st_qwenturbo8k_reasoning_config
                    )

此方法将PDF报告解析为JSON文件。它会在debug/data_01_parsed_reports目录中创建JSON文件。这些JSON文件将在后续步骤中使用  
它还会将文档生成的原始输出存储在 debug/data_01_parsed_reports_debug目录中。这些JSON文件包含大量元数据，但不会被使用  
在这一步，会调用多模态大模型生成图片描述  

In [ ]:
import os

# 设置镜像地址
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
pipeline.parse_pdf_reports_sequential()

此方法将debug/data_01_parsed_reports中的JSON转换为更简单的JSON，即Markdown格式的页面列表
新的JSON文件位于debug/data_02_merged_reports中

In [ ]:
pipeline.merge_reports()

此方法将报告导出为纯 Markdown 格式。这些报告仅用于审阅和全文搜索配置：gemini_thinking_config
新文件位于 debug/data_03_reports_markdown 目录下

In [ ]:
pipeline.export_reports_to_markdown()

此方法将报告分割成多个数据块，用于向量化处理
新的 JSON 文件位于 databases/chunked_reports 目录中。

In [ ]:
# TODO 在这一步调用qwen向量模型api时，并发请求速率过快，会被拦截
pipeline.chunk_reports()

此方法从分块报告中创建向量数据库
新文件位于 databases/vector_dbs 目录中。

In [ ]:
pipeline.create_vector_dbs()

此方法处理问题和答案
问题处理逻辑取决于run_config

In [ ]:
pipeline.process_questions()